In [248]:
import pandas as pd
import numpy as np
jobs = ['Data scientist', 'Python', 'JS', 'Java','Data scientist', 'Python', 'JS', 'Java']
countrys = ['France','France','France','France', 'USA','USA','USA','USA']
filenames = ['data/'+str.lower(job).replace(' ','')+str.lower(country)+'.csv' for job, country in zip(jobs,countrys) ]
df = pd.concat([pd.read_csv(filename, sep=';', encoding="utf-8-sig", index_col=0) for filename in filenames])

In [239]:
df.head()

,Entreprise,Pays,Ville,Type de contrat,Salaire,Description
Poste,,,,,,
Alternance - Data Scientist H/F,Malakoff Humanis,France,Paris 9e (75),"Apprentissage, Contrat pro",None,Contrat ALTERNANCE Finalité du poste Au sein d...
Data Scientist H/F,AGYLIS,France,La Défense (92),"Temps plein, CDI",40 000 € - 50 000 € par an,AGYLIS est un acteur important des Entreprise ...
DATA SCIENTIST- F/H,Ingeniance,France,La Défense (92),CDI,None,"Contexte: Vous assurez la production, la conce..."
Data Scientist and Innovation Engineer H/F,HIPAY,France,Levallois-Perret (92),None,None,Notre métier HiPay est une Fintech Européenne ...
ASSISTANT DATA SCIENTIST,Total,France,Nanterre (92),"Apprentissage, Contrat pro",None,Monter un système de reporting pour la directi...


In [249]:
df2 = df.drop_duplicates(subset=None, keep="first")

In [257]:
skills = ['Python', 'R', 'SQL', 'NoSQL', 'GIT', 'Spark', 'Flask', 'Streamlit', 'Docker', 'Kubernetes', 'React', 'Machine-Learning', 'Deep Learning', 'NLP', 'Vue', 'Angular', 'Scala', 'PySpark','PowerBI', 'SQLServer', 'Dataiku', 'Keras', 'Tensor-Flow', 'NLU', 'PyTorch', 'Scikit-Learn', 'SAS', 'Java', 'Scikit learn', 'Hadoop', 'Hive', 'ML','DL', 'Azure', 'AWS']

contrats = ['CDI','CDD','Apprentissage','Contrat pro','Stage','Intérim','Freelance','Indépendant','Freelance']
            
explevels = ['1 year','1+ year','1-2 years','1-3+ years','2 years','2+ years','two (2) years','4 years','3 years','4+ years','5 years','5+ years','10 years','4-6+ years']

studylevels = ['formation supérieure','Diplôme ingénieur','Master', 'Master 2','Bac +2','Bac +3','Bac +4','Bac +5','grande école d\'ingénieur','Bac +4/5','M2','Cursus ingénieur','formation Data Science ou IA','Formation Data Science ou IA','Formation Data Science','Université','Universitaire']

worktimes = ['Full-time','Part-time','Remote','Internship','Contract','Temporary','Comission']

worktime = ['Temps plein','Temps partiel']

langages = ['Spanish','Chinese','Arab','French','Italian']

usaSchoolLevels = ['High School','high school','College','Graduate','Master Degree','University','university','master','PhD','Associate\'s degree','Bachelor Degree','Associate','Bachelor','Baccalaureate','baccalaureate','Bachelor of Science']

salaries = ['\$(\d){2,3}\,(\d){3}( \- )\$(\d){2,3}\,(\d){3}( a year)|\$(\d){2,3}\,(\d){3}( a year)|\$(\d){2}\.(\d){2}( \- )\$(\d){2}\.(\d){2}( an hour)|\$(\d){2}\.(\d){2}( an hour)']


In [251]:
df2 = pd.concat([df2, pd.DataFrame(columns=skills)])

In [260]:
def study_level_required(x):
    for level in studylevels:
        if level  in str.lower(x) or str.lower(level) in str.lower(x) or str.lower(level.replace(' ','')) in str.lower(x):
            return level
    return np.NaN

df2['Study level required'] = df2['Description'].apply(lambda x : study_level_required(x))

In [253]:
df2['Study level required'].isna().sum()

2550

In [261]:
def skill_required(row):

    desc_low = str.lower(row['Description'])
    
    for skill in skills:

        skill_low = str.lower(skill)

        if (skill_low in desc_low) or (skill_low.replace(' ','') in desc_low) or (skill_low.replace('-','') in desc_low)  or (skill_low.replace('-',' ') in desc_low):
            row[skill] = 1

    return row

In [262]:
df2 = df2.apply(lambda row: skill_required(row), axis=1)

In [263]:
df2

,Entreprise,Pays,Ville,Type de contrat,Salaire,Description,Python,R,SQL,NoSQL,...,SAS,Java,Scikit learn,Hadoop,Hive,ML,DL,Azure,AWS,Study level required
Alternance - Data Scientist H/F,Malakoff Humanis,France,Paris 9e (75),"Apprentissage, Contrat pro",None,Contrat ALTERNANCE Finalité du poste Au sein d...,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bac +5
Data Scientist H/F,AGYLIS,France,La Défense (92),"Temps plein, CDI",40 000 € - 50 000 € par an,AGYLIS est un acteur important des Entreprise ...,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,Master
DATA SCIENTIST- F/H,Ingeniance,France,La Défense (92),CDI,None,"Contexte: Vous assurez la production, la conce...",NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bac +5
Data Scientist and Innovation Engineer H/F,HIPAY,France,Levallois-Perret (92),None,None,Notre métier HiPay est une Fintech Européenne ...,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ASSISTANT DATA SCIENTIST,Total,France,Nanterre (92),"Apprentissage, Contrat pro",None,Monter un système de reporting pour la directi...,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bac +5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Senior Developer,VirtualMGA,USA,United States,None,None,Virtual MGA is searching for someone who can w...,NaN,1.0,1.0,NaN,...,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
"Senior Backend Engineer, Remote Assistance",Cruise,USA,"San Francisco, CA 94103 (South of Market area)",None,None,"We're Cruise, the self-driving ride-hailing se...",NaN,1.0,1.0,1.0,...,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,Master
Principal Software Engineer/Architect,Microsoft,USA,"Redmond, WA",None,None,Are you interested in joining a startup within...,NaN,1.0,1.0,NaN,...,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN
Sr. Software Engineer,"Homecare Homebase, LLC",USA,"Louisville, KY",None,None,Position Summary A Senior Software Engineer wo...,NaN,1.0,1.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
df2.iloc[2][5]

"Contexte: Vous assurez la production, la conception et la mise en oeuvre des nouvelles solutions ainsi que la définition des standards et de la stratégie et serez également en charge du développement de l'Intelligence Artificielle et de la Data Science appliquée à l'infrastructure informatique d'une grande banque d'investissement.  Missions: Formaliser des problématiques business (infrastructure IT) en projet de data science Proposer des évolutions des produits existants ou de nouveaux produits Contribuer au développement de MVP et de data products de production Contribuer au suivi et à l'amélioration continue des data products en production Proposer et mettre en oeuvre l'industrialisation du machine Learning  Environnement: Scoping des projets et définition des portefeuilles de projets Data Science Suivi et réalisation des projets de data science Maquettes ou MVPs Bonnes pratiques data science Pipelines industriels de machine learning Profil recherché: Diplômé(e) d'une grande école d

Master
